## LightGBM  (5) 
- player情報を付加
- 2017年の実績を付加
- 特徴量数: 135

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

In [11]:
ALL_MERGE = 'intermediate/all_merge_5.f'

In [12]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 150)

### 不要な列を削除

In [13]:
all_pitch.drop(
    columns=[
        '年度', 
        '試合ID', 
        'ホームチームID', 'アウェイチームID', 
        '投手ID', '投手チームID', 
        '打者ID', '打者チームID', 
        'プレイ前走者状況', 
        '捕手ID', 
        'opening_date', 'game_date',
        'start_time', 'game_time', 'elapsed_time'
    ], inplace=True)

### Rename

In [14]:
all_pitch.rename(columns={
    'データ内連番': 'No',
    '投球位置区域': 'pitch_area',
    '試合内連番': 'No_in_game',
    '試合内投球数': 'pitch_cnt_in_game',
    'イニング': 'inning',
    'イニング内打席数': 'bat_cnt_in_inning',
    '打席内投球数': 'pitch_cnt_in_bat',
    '投手登板順': 'pitch_order',
    '投手試合内対戦打者数': 'player_cnt_in_game',
    '投手試合内投球数': 'pitcher_cnt_in_game',
    '投手イニング内投球数': 'pitcher_cnt_in_inning',
    '打者打順': 'bat_order',
    '打者試合内打席数': 'bat_cnt_in_game',
    'プレイ前ホームチーム得点数': 'home_point',
    'プレイ前アウェイチーム得点数': 'away_point',
    'プレイ前アウト数': 'out_cnt',
    'プレイ前ボール数': 'ball_cnt',
    'プレイ前ストライク数': 'strike_cnt',
}, inplace=True)

In [15]:
print(all_pitch.shape)
all_pitch.head()

(778767, 135)


,No,ball,pitch_area,No_in_game,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,first,second,third,base_cnt,pitch_LR,bat_LR,top_bot,role,pos_pit,date_from_opening,elapsed_min,nighter,ce-pa,league,home,point_diff,runner_23,bant,squize,cleanup,pinch,fourball,savepoint,ball_cnt_0-0,ball_cnt_0-1,ball_cnt_0-2,ball_cnt_0-3,ball_cnt_1-0,ball_cnt_1-1,ball_cnt_1-2,ball_cnt_1-3,...,firm_bat,height_bat,weight_bat,draft_order_bat,salary_bat,foreigner_bat,batter_cnt_bat,bat_game_cnt_bat,company_bat,univ_bat,highsch_bat,age_bat,play_year_bat,salary_year_bat,salary_x_year_bat,bmi_bat,firm,height,weight,draft_order,salary,foreigner,batter_cnt,bat_game_cnt,company,univ,highsch,age,play_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,bmi_dif_b-c
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0.0,1.0,1,1,0,0,0.000000,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
1,1,0.0,8.0,2,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0,0,0,0,0.0,1.0,1,1,0,0,0.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
2,2,0.0,5.0,3,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0,0,0,0,0.0,1.0,1,1,0,0,0.383333,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,176,74,5.0,15000,0,523.0,119.0,1,1,0,32,8.0,1875.0,120000.0,23.889463,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,2.141358,19000,2.0,1,2750.0,124000.0,-0.165367,10000,4.0,4,625.0,100000.0,-2.306725
3,3,0.0,12.0,4,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0,0,0,0,0.0,0.0,1,1,0,0,1.016667,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681
4,4,0.0,8.0,5,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0,0,0,0,0.0,0.0,1,1,0,0,1.200000,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,...,0,180,74,1.0,7040,0,268.0,81.0,0,0,1,40,22.0,320.0,154880.0,22.839506,0,178,83,1.0,5000,0,443.0,135.0,1,1,0,28,4.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,3.191314,19000,2.0,1,2750.0,124000.0,-0.165367,2040,18.0,12,-930.0,134880.0,-3.356681


### train
- 行数: 257117

In [16]:
train = all_pitch.dropna(subset=['pitch_area'])
train.shape

(257117, 135)

### test
- 行数: 521650

In [17]:
test = all_pitch[all_pitch['pitch_area'].isnull()]
test.shape

(521650, 135)

In [18]:
del all_pitch
gc.collect()

64

In [19]:
train_d = train.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

In [20]:
test_d = test.drop([
    'No', 
    'pitch_area', 
    'ball'
], axis=1)

## Dataset作成

In [21]:
lgb_train = lgb.Dataset(train_d, train['ball'])

### LGBM学習
- multiclass : クラス数=8

In [22]:
%%time
lgb_param = {
        'objective' : 'multiclass',
        'boosting_type': 'gbdt',
        'metric' : 'multi_logloss',
        'num_class' : 8,
        'num_leaves' : 31,
        'seed' : 0,
        'learning_rate' : 0.1,
    }

# 学習
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=8000,
                    early_stopping_rounds=100,
                    verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['multi_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
print('Best CV score:', cv_results['multi_logloss-mean'][-1])

[100]	cv_agg's multi_logloss: 1.12978 + 0.00197043
[200]	cv_agg's multi_logloss: 1.12083 + 0.00244572
[300]	cv_agg's multi_logloss: 1.11765 + 0.00264929
[400]	cv_agg's multi_logloss: 1.11638 + 0.00277725
[500]	cv_agg's multi_logloss: 1.11628 + 0.00303695
Best num_boost_round: 443
Best CV score: 1.1161825304353852
CPU times: user 59min 32s, sys: 34 s, total: 1h 6s
Wall time: 15min 47s


In [23]:
best_iter = int(num_boost_round * 1.1)

### 全体で再学習

In [24]:
%%time
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

CPU times: user 18min 6s, sys: 20.6 s, total: 18min 26s
Wall time: 6min 8s


### Feature Importance

In [25]:
fi = lgb_model.feature_importance()
fn = lgb_model.feature_name()
df_feature_importance = pd.DataFrame({'feat_name':fn, 'feat_imp':fi})
df_feature_importance.sort_values('feat_imp', inplace=True)
df_feature_importance.tail(30)

,feat_name,feat_imp
126,salary_dif_b-c,1391
6,player_cnt_in_game,1397
130,salary_x_year_dif_b-c,1398
95,salary_year_bat,1408
114,salary_dif_p-b,1426
118,salary_x_year_dif_p-b,1426
84,weight_bat,1530
64,changeup_pit,1543
62,shoot_pit,1570
89,bat_game_cnt_bat,1606


In [26]:
%%time
predict = lgb_model.predict(test_d, num_iteration = lgb_model.best_iteration)

CPU times: user 8min 41s, sys: 5.53 s, total: 8min 47s
Wall time: 3min 11s


In [27]:
predict

array([[3.00504877e-01, 2.08421849e-02, 9.11404532e-02, ...,
        8.54933723e-08, 1.26965568e-06, 2.27114954e-01],
       [2.20038291e-01, 5.71224804e-02, 8.78444365e-02, ...,
        1.23733741e-07, 1.77970741e-07, 4.12840926e-02],
       [2.90858891e-01, 9.20778551e-02, 1.42474032e-01, ...,
        3.71981651e-07, 1.16861637e-06, 1.08080982e-01],
       ...,
       [5.36934594e-01, 5.83376977e-02, 3.87310079e-04, ...,
        3.01813519e-07, 7.70756017e-07, 1.27173673e-03],
       [6.21930609e-01, 2.93310996e-02, 3.04422039e-04, ...,
        2.96925991e-07, 1.05892627e-06, 8.87869626e-04],
       [6.01410166e-01, 3.91081508e-02, 3.31661339e-04, ...,
        3.16125355e-07, 1.15880428e-06, 8.69314221e-04]])

## 結果出力

In [28]:
submit = pd.DataFrame(predict)
submit.reset_index(inplace=True)
print(submit.shape)
submit.head()

(521650, 9)


,index,0,1,2,3,4,5,6,7
0,0,0.300505,0.020842,0.091140,0.358811,0.001585,8.549337e-08,1.269656e-06,0.227115
1,1,0.220038,0.057122,0.087844,0.589574,0.004137,1.237337e-07,1.779707e-07,0.041284
2,2,0.290859,0.092078,0.142474,0.274865,0.091642,3.719817e-07,1.168616e-06,0.108081
3,3,0.219115,0.070789,0.117922,0.525744,0.010860,2.245924e-07,3.147817e-07,0.055569
4,4,0.147093,0.032418,0.166625,0.503100,0.006914,2.039212e-07,6.483916e-07,0.143848


### 2017の平均
- ストレート           0.468254
- カーブ             0.074725
- スライダー           0.185806
- シュート            0.070633
- フォーク            0.083013
- チェンジアップ         0.054217
- シンカー            0.011361
- カットボール          0.051992

In [29]:
submit.mean()

index    260824.500000
0             0.493382
1             0.075131
2             0.182564
3             0.062317
4             0.075422
5             0.051083
6             0.008457
7             0.051644
dtype: float64

In [30]:
submit.to_csv('submit/lgbm_5.csv', header=False, index=False)

### 評価結果
#### 5 (135)
- CV score: 1.1161825304353852
- 評価結果  : 1.4252286

#### 4 (128)
- CV score: 1.1158111693845418
- 評価結果  : 1.4299265

#### 3 (97)
- CV score: 1.1172290313076036
- 評価結果  : 1.4335806